# DFT

In [8]:
inputs = ['../data/augment_data_from_csvs/',
          '../data/augment_data_from_csvs/no_interm/',
]

outputs = ['../results/csv_based/',
           '../results/csv_without_interms/',
]

ind = 1

withoutInterms = (ind == 1)

output_root = outputs[ind]
input_root = inputs[ind]

if withoutInterms:
    print('Intermediate user data will be skipped')
else:
    print('All three user groups (Beginner, Intermediate, Expert) are processed')
    
groupRange = [0,2]
if not withoutInterms:
    groupRange = [0,1,2]
print(groupRange)

import os.path
from os import path

generateOnlyMissingResultFiles = True

Intermediate user data will be skipped
[0, 2]


In [9]:
# -*- coding: utf-8 -*-
"""
Created on Wed Apr 17 17:00:02 2019

@author: xngu0004
"""

import numpy as np
import random
import scipy.io as sio
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn import svm
from scipy.fftpack import fft

list_skill = ["Suture", "Knot", "Needle"]
list_win = [90, 120, 150, 180]
list_over = [30, 60]
list_netw = [1]

# users in Group
novice = ['B', 'G', 'H', 'I']
intermediate = ['C', 'F']
expert = ['D', 'E']

f_s = 30 # Sampling rate
denominator = 10

# Function to calculate FFT coefficiences
def get_fft_values(y_values, T, N, f_s):
    f_values = np.linspace(0.0, 1.0/(T), N)
    fft_values_ = fft(y_values)
    fft_values = 1.0/N * np.abs(fft_values_[0:N//2])
    return f_values, fft_values

# Revised from http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb
def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising', kpsh=False, valley=False, ax=None):

    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan-1, indnan+1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size-1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind]-x[ind-1], x[ind]-x[ind+1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                    & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indices by their occurrence
        ind = np.sort(ind[~idel])

    return ind

# Extract no_peaks
def get_first_n_peaks(x,y,no_peaks=10):
    x_, y_ = list(x), list(y)
    if len(x_) >= no_peaks:
        return x_[:no_peaks], y_[:no_peaks]
    else:
        missing_no_peaks = no_peaks-len(x_)
        return x_ + [0]*missing_no_peaks, y_ + [0]*missing_no_peaks
    
def get_features(x_values, y_values, mph):
    indices_peaks = detect_peaks(y_values, mph=mph)
    peaks_x, peaks_y = get_first_n_peaks(x_values[indices_peaks], y_values[indices_peaks])
    return peaks_x + peaks_y

# Extract features
def extract_features_labels(dataset, labels, T, N, f_s, denominator):
    percentile = 5
    list_of_features = []
    list_of_labels = []
    for signal_no in range(0, len(dataset)):
        features = []
        list_of_labels.append(labels[signal_no])
        for signal_comp in range(0,dataset.shape[2]):
            signal = dataset[signal_no, :, signal_comp]
            
            signal_min = np.nanpercentile(signal, percentile)
            signal_max = np.nanpercentile(signal, 100-percentile)
            ##ijk = (100 - 2*percentile)/10
            mph = signal_min + (signal_max - signal_min)/denominator
            
            #features += get_features(*get_psd_values(signal, T, N, f_s), mph)
            features += get_features(*get_fft_values(signal, T, N, f_s), mph)
            #features += get_features(*get_autocorr_values(signal, T, N, f_s), mph)
        list_of_features.append(features)
    return np.array(list_of_features), np.array(list_of_labels)

import os.path
from os import path

for win in list_win:
    for over in list_over:
        f_s = over #(it is set to 30 by default, but I noticed that maybe it needs to switch to 60 when over is 60)
        for skill in list_skill:
            input_path = input_root+skill+'_dataX_basic_'+str(win)+'_'+str(over)+'.mat'
            " Load the data"
            print('loading the dataset')
            dataset = sio.loadmat(input_path)
            print(input_path, ' loaded')
            features = dataset['dataX']
            sizeD_ts = (((features[0,0:1])[0])[0,:])[0].shape[0]
            sizeD_va = (((features[0,0:1])[0])[0,:])[0].shape[1]
            N = sizeD_ts # Number of samples
            t_n = N/f_s # Duration of window
            T = 1 / f_s # Period
            print('loaded the dataset')
            for netw in list_netw:
                for tim in range(3,4):
                    full_output_path = output_root + 'DFT/' + str(tim)+'_'+skill+'_'+str(win)+'_'+str(over)+'_'+str(netw)+'_report_DFT_fea.txt'
                    if generateOnlyMissingResultFiles and path.exists(full_output_path):
                        print(full_output_path + ' already exists')
                        continue
                    acc_1 = []
                    accu = 0
                    acc_1_fea = []
                    accu_fea = 0                    
                    ###################################
                    #Pre-process data
                    for user in range(1,6):
                        " Initialize data for concatenation "
                        X_test1 = np.ones((1,sizeD_ts,sizeD_va))
                        y_test1 = np.ones(1)
                        X_train1 = np.ones((1,sizeD_ts,sizeD_va))
                        y_train1 = np.ones(1)
                        print("User out: ", user)
                        # Extract data for training and testing
                        for j in groupRange:
                            upper_range =  (((features[0,j:j+1])[0])[0,:]).shape[0]
                            for i in range(0, upper_range):
                                k = (((features[0,j:j+1])[0])[2,:])[i]
                                if (k == user): # take out the ith trial of each user for testing
                                    temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                                    temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                                    X_test1 = np.concatenate((X_test1, temp_x1), axis=0) 
                                    user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                                    if(user_temp in novice):
                                        y_test1 = np.concatenate((y_test1,[0]))
                                    elif(user_temp in intermediate):
                                        y_test1 = np.concatenate((y_test1,[1]))
                                    elif(user_temp in expert):
                                        y_test1 = np.concatenate((y_test1,[2]))
                                else:
                                    temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                                    temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                                    X_train1 = np.concatenate((X_train1, temp_x1), axis=0) 
                                    user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                                    if(user_temp in novice):
                                        y_train1 = np.concatenate((y_train1,[0]))
                                    elif(user_temp in intermediate):
                                        y_train1 = np.concatenate((y_train1,[1]))
                                    elif(user_temp in expert):
                                        y_train1 = np.concatenate((y_train1,[2]))
        
                        " Delete the first column "
                        X_test1 = np.delete(X_test1,np.s_[0:1], axis=0)
                        y_test1 = np.delete(y_test1,np.s_[0:1])
                        X_train1 = np.delete(X_train1,np.s_[0:1], axis=0)
                        y_train1= np.delete(y_train1,np.s_[0:1])
                        
                        print("Shuffle data")
                        " Shuffle the training and testing data "
                        Xx_test, yy_test = shuffle(X_test1, y_test1, random_state = random.randint(10,50))
                        Xx_train, yy_train = shuffle(X_train1, y_train1, random_state = random.randint(10,50))
            
                        ##############################################
                        X_train, Y_train = extract_features_labels(Xx_train, yy_train, T, N, f_s, denominator)
                        X_test, Y_test = extract_features_labels(Xx_test, yy_test, T, N, f_s, denominator)
                        ##############################################
                        
                        svm1 = svm.SVC(kernel = 'linear')
                        svm1.fit(X_train, Y_train)
                        prediction_fea = svm1.predict(X_test)
                        y_pred_fea = prediction_fea
                        cr_fea = classification_report(Y_test, y_pred_fea)
                        cm_fea = confusion_matrix(Y_test, y_pred_fea)
    
                        acc_fea = np.sum(y_pred_fea == Y_test)/y_pred_fea.shape[0]
                        acc_1_fea.append(acc_fea)
                        print('Accuracy: ' + str(acc_fea))
                        print(cr_fea)
                        print(cm_fea)
                        f = open(full_output_path, 'a+')
                        f.write("-------------------- DFT Features -----------------------")
                        f.write('---------' + skill + " Win: " + str(win) + " Over: " + str(over) + " Trial: " + str(user) + '---------\n\n')
                        f.write('Accuracy: ' + str(acc_fea))
                        f.write('\n\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n\n'.format(cr_fea, cm_fea))
                    for i in range(0,5):
                        accu_fea += acc_1_fea[i]
                    accu_fea = accu_fea/5
                    f.write("\n-------------------------------------------\n")
                    f.write("\nAcc_features = " + str(accu_fea))
                    f.write("\n-------------------------------------------\n")
                    f.close()

loading the dataset
../data/augment_data_from_csvs/no_interm/Suture_dataX_basic_90_30.mat  loaded
loaded the dataset
../results/csv_without_interms/DFT/3_Suture_90_30_1_report_DFT_fea.txt already exists
loading the dataset
../data/augment_data_from_csvs/no_interm/Knot_dataX_basic_90_30.mat  loaded
loaded the dataset
../results/csv_without_interms/DFT/3_Knot_90_30_1_report_DFT_fea.txt already exists
loading the dataset
../data/augment_data_from_csvs/no_interm/Needle_dataX_basic_90_30.mat  loaded
loaded the dataset
User out:  1
Shuffle data
Accuracy: 0.5092592592592593
              precision    recall  f1-score   support

         0.0       0.33      0.93      0.48        80
         2.0       0.94      0.37      0.53       244

    accuracy                           0.51       324
   macro avg       0.63      0.65      0.51       324
weighted avg       0.79      0.51      0.52       324

[[ 74   6]
 [153  91]]
User out:  2
Shuffle data
Accuracy: 0.5226130653266332
              precisi

Shuffle data
Accuracy: 0.5
              precision    recall  f1-score   support

         0.0       0.52      0.85      0.65        94
         2.0       0.36      0.10      0.15        82

    accuracy                           0.50       176
   macro avg       0.44      0.47      0.40       176
weighted avg       0.45      0.50      0.42       176

[[80 14]
 [74  8]]
loading the dataset
../data/augment_data_from_csvs/no_interm/Suture_dataX_basic_120_30.mat  loaded
loaded the dataset
User out:  1
Shuffle data
Accuracy: 0.4882533197139939
              precision    recall  f1-score   support

         0.0       0.73      0.57      0.64       777
         2.0       0.10      0.18      0.13       202

    accuracy                           0.49       979
   macro avg       0.41      0.37      0.38       979
weighted avg       0.60      0.49      0.53       979

[[442 335]
 [166  36]]
User out:  2
Shuffle data
Accuracy: 0.5681818181818182
              precision    recall  f1-score   sup

Shuffle data
Accuracy: 0.39325842696629215
              precision    recall  f1-score   support

         0.0       0.63      0.46      0.53       199
         2.0       0.11      0.19      0.14        68

    accuracy                           0.39       267
   macro avg       0.37      0.33      0.34       267
weighted avg       0.49      0.39      0.43       267

[[ 92 107]
 [ 55  13]]
loading the dataset
../data/augment_data_from_csvs/no_interm/Knot_dataX_basic_120_60.mat  loaded
loaded the dataset
User out:  1
Shuffle data
Accuracy: 0.5174825174825175
              precision    recall  f1-score   support

         0.0       0.67      0.60      0.63        99
         2.0       0.27      0.34      0.30        44

    accuracy                           0.52       143
   macro avg       0.47      0.47      0.47       143
weighted avg       0.55      0.52      0.53       143

[[59 40]
 [29 15]]
User out:  2
Shuffle data
Accuracy: 0.5333333333333333
              precision    recall  

Shuffle data
Accuracy: 0.6245353159851301
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74       196
         2.0       0.32      0.33      0.32        73

    accuracy                           0.62       269
   macro avg       0.53      0.53      0.53       269
weighted avg       0.63      0.62      0.63       269

[[144  52]
 [ 49  24]]
loading the dataset
../data/augment_data_from_csvs/no_interm/Needle_dataX_basic_150_30.mat  loaded
loaded the dataset
User out:  1
Shuffle data
Accuracy: 0.5283018867924528
              precision    recall  f1-score   support

         0.0       0.33      0.94      0.49        78
         2.0       0.95      0.40      0.56       240

    accuracy                           0.53       318
   macro avg       0.64      0.67      0.53       318
weighted avg       0.80      0.53      0.54       318

[[ 73   5]
 [145  95]]
User out:  2
Shuffle data
Accuracy: 0.5256410256410257
              precision    rec

Shuffle data
Accuracy: 0.5232558139534884
              precision    recall  f1-score   support

         0.0       0.53      0.87      0.66        92
         2.0       0.45      0.12      0.20        80

    accuracy                           0.52       172
   macro avg       0.49      0.50      0.43       172
weighted avg       0.50      0.52      0.44       172

[[80 12]
 [70 10]]
loading the dataset
../data/augment_data_from_csvs/no_interm/Suture_dataX_basic_180_30.mat  loaded
loaded the dataset
User out:  1
Shuffle data
Accuracy: 0.5015511892450879
              precision    recall  f1-score   support

         0.0       0.73      0.59      0.65       769
         2.0       0.09      0.16      0.12       198

    accuracy                           0.50       967
   macro avg       0.41      0.38      0.38       967
weighted avg       0.60      0.50      0.54       967

[[453 316]
 [166  32]]
User out:  2
Shuffle data
Accuracy: 0.6428571428571429
              precision    recall 

Shuffle data
Accuracy: 0.24521072796934865
              precision    recall  f1-score   support

         0.0       0.49      0.28      0.36       195
         2.0       0.06      0.14      0.08        66

    accuracy                           0.25       261
   macro avg       0.28      0.21      0.22       261
weighted avg       0.38      0.25      0.29       261

[[ 55 140]
 [ 57   9]]
loading the dataset
../data/augment_data_from_csvs/no_interm/Knot_dataX_basic_180_60.mat  loaded
loaded the dataset
User out:  1
Shuffle data
Accuracy: 0.6594202898550725
              precision    recall  f1-score   support

         0.0       0.81      0.67      0.73        96
         2.0       0.46      0.64      0.53        42

    accuracy                           0.66       138
   macro avg       0.63      0.65      0.63       138
weighted avg       0.70      0.66      0.67       138

[[64 32]
 [15 27]]
User out:  2
Shuffle data
Accuracy: 0.7
              precision    recall  f1-score   supp

# DCT

In [6]:
inputs = ['../data/augment_data_from_csvs/',
          '../data/augment_data_from_csvs/no_interm/',
]

outputs = ['../results/csv_based/',
           '../results/csv_without_interms/',
]

ind = 1

withoutInterms = (ind == 1)

output_root = outputs[ind]
input_root = inputs[ind]

if withoutInterms:
    print('Intermediate user data will be skipped')
else:
    print('All three user groups (Beginner, Intermediate, Expert) are processed')
    
groupRange = [0,2]
if not withoutInterms:
    groupRange = [0,1,2]
print(groupRange)

import os.path
from os import path

generateOnlyMissingResultFiles = True

Intermediate user data will be skipped
[0, 2]


In [8]:
# -*- coding: utf-8 -*-
"""
Created on Wed Apr 17 17:00:02 2019

@author: xngu0004
"""

import numpy as np
import random
import scipy.io as sio
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn import svm
from scipy.fftpack import dct

list_skill = ["Suture", "Knot", "Needle"]
list_win = [90, 120, 150, 180]
list_over = [30, 60]
list_netw = [1]

# users in Group
novice = ['B', 'G', 'H', 'I']
intermediate = ['C', 'F']
expert = ['D', 'E']

f_s = 30 # Sampling rate
denominator = 10

# Function to calculate FFT coefficiences
def get_dct_values(y_values, T, N, f_s):
    f_values = np.linspace(0.0, 1.0/(T), N)
    dct_values_ = dct(y_values,3)
    dct_values = 1.0/N * np.abs(dct_values_[0:N//2])
    return f_values, dct_values

# Revised from http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb
def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising', kpsh=False, valley=False, ax=None):

    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan-1, indnan+1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size-1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind]-x[ind-1], x[ind]-x[ind+1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                    & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indices by their occurrence
        ind = np.sort(ind[~idel])

    return ind

# Extract no_peaks
def get_first_n_peaks(x,y,no_peaks=10):
    x_, y_ = list(x), list(y)
    if len(x_) >= no_peaks:
        return x_[:no_peaks], y_[:no_peaks]
    else:
        missing_no_peaks = no_peaks-len(x_)
        return x_ + [0]*missing_no_peaks, y_ + [0]*missing_no_peaks
    
def get_features(x_values, y_values, mph):
    indices_peaks = detect_peaks(y_values, mph=mph)
    peaks_x, peaks_y = get_first_n_peaks(x_values[indices_peaks], y_values[indices_peaks])
    return peaks_x + peaks_y

# Extract features
def extract_features_labels(dataset, labels, T, N, f_s, denominator):
    percentile = 5
    list_of_features = []
    list_of_labels = []
    for signal_no in range(0, len(dataset)):
        features = []
        list_of_labels.append(labels[signal_no])
        for signal_comp in range(0,dataset.shape[2]):
            signal = dataset[signal_no, :, signal_comp]
            
            signal_min = np.nanpercentile(signal, percentile)
            signal_max = np.nanpercentile(signal, 100-percentile)
            ##ijk = (100 - 2*percentile)/10
            mph = signal_min + (signal_max - signal_min)/denominator
            
            #features += get_features(*get_psd_values(signal, T, N, f_s), mph)
            #features += get_features(*get_fft_values(signal, T, N, f_s), mph)
            features += get_features(*get_dct_values(signal, T, N, f_s), mph)
            #features += get_features(*get_autocorr_values(signal, T, N, f_s), mph)
        list_of_features.append(features)
    return np.array(list_of_features), np.array(list_of_labels)

for win in list_win:
    for over in list_over:
        f_s = over #(it is set to 30 by default, but I noticed that maybe it needs to switch to 60 when over is 60)
        for skill in list_skill:
            " Load the data"
            print('loading the dataset')
            dataset = sio.loadmat(input_root + skill+'_dataX_basic_'+str(win)+'_'+str(over)+'.mat')
            features = dataset['dataX']
            sizeD_ts = (((features[0,0:1])[0])[0,:])[0].shape[0]
            sizeD_va = (((features[0,0:1])[0])[0,:])[0].shape[1]
            N = sizeD_ts # Number of samples
            t_n = N/f_s # Duration of window
            T = 1 / f_s # Period
            print('loaded the dataset')
            for netw in list_netw:
                for tim in range(3,5):
                    full_output_path = output_root + 'DCT/' + str(tim)+'_'+skill+'_'+str(win)+'_'+str(over)+'_'+str(netw)+'_report_DCT_fea.txt'
                    if generateOnlyMissingResultFiles and path.exists(full_output_path):
                        print(full_output_path + ' already exists')
                        continue
                    acc_1 = []
                    accu = 0
                    acc_1_fea = []
                    accu_fea = 0                    
                    ###################################
                    #Pre-process data
                    for user in range(1,6):
                        " Initialize data for concatenation "
                        X_test1 = np.ones((1,sizeD_ts,sizeD_va))
                        y_test1 = np.ones(1)
                        X_train1 = np.ones((1,sizeD_ts,sizeD_va))
                        y_train1 = np.ones(1)
                        print("User out: ", user)
                        # Extract data for training and testing
                        for j in groupRange:
                            for i in range(0, (((features[0,j:j+1])[0])[0,:]).shape[0]):
                                k = (((features[0,j:j+1])[0])[2,:])[i]
                                if (k == user): # take out the ith trial of each user for testing
                                    temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                                    temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                                    X_test1 = np.concatenate((X_test1, temp_x1), axis=0) 
                                    user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                                    if(user_temp in novice):
                                        y_test1 = np.concatenate((y_test1,[0]))
                                    elif(user_temp in intermediate):
                                        y_test1 = np.concatenate((y_test1,[1]))
                                    elif(user_temp in expert):
                                        y_test1 = np.concatenate((y_test1,[2]))
                                else:
                                    temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                                    temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                                    X_train1 = np.concatenate((X_train1, temp_x1), axis=0) 
                                    user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                                    if(user_temp in novice):
                                        y_train1 = np.concatenate((y_train1,[0]))
                                    elif(user_temp in intermediate):
                                        y_train1 = np.concatenate((y_train1,[1]))
                                    elif(user_temp in expert):
                                        y_train1 = np.concatenate((y_train1,[2]))
        
                        " Delete the first column "
                        X_test1 = np.delete(X_test1,np.s_[0:1], axis=0)
                        y_test1 = np.delete(y_test1,np.s_[0:1])
                        X_train1 = np.delete(X_train1,np.s_[0:1], axis=0)
                        y_train1= np.delete(y_train1,np.s_[0:1])
                        
                        print("Shuffle data")
                        " Shuffle the training and testing data "
                        Xx_test, yy_test = shuffle(X_test1, y_test1, random_state = random.randint(10,50))
                        Xx_train, yy_train = shuffle(X_train1, y_train1, random_state = random.randint(10,50))
            
                        ##############################################
                        X_train, Y_train = extract_features_labels(Xx_train, yy_train, T, N, f_s, denominator)
                        X_test, Y_test = extract_features_labels(Xx_test, yy_test, T, N, f_s, denominator)
                        ##############################################
                        
                        svm1 = svm.SVC(kernel = 'linear')
                        svm1.fit(X_train, Y_train)
                        prediction_fea = svm1.predict(X_test)
                        y_pred_fea = prediction_fea
                        cr_fea = classification_report(Y_test, y_pred_fea)
                        cm_fea = confusion_matrix(Y_test, y_pred_fea)
    
                        acc_fea = np.sum(y_pred_fea == Y_test)/y_pred_fea.shape[0]
                        acc_1_fea.append(acc_fea)
                        print('Accuracy: ' + str(acc_fea))
                        print(cr_fea)
                        print(cm_fea)
                        f = open(full_output_path, 'a+')
                        f.write("-------------------- DCT Features -----------------------")
                        f.write('---------' + skill + " Win: " + str(win) + " Over: " + str(over) + " Trial: " + str(user) + '---------\n\n')
                        f.write('Accuracy: ' + str(acc_fea))
                        f.write('\n\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n\n'.format(cr_fea, cm_fea))
                    for i in range(0,5):
                        accu_fea += acc_1_fea[i]
                    accu_fea = accu_fea/5
                    f.write("\n-------------------------------------------\n")
                    f.write("\nAcc_features = " + str(accu_fea))
                    f.write("\n-------------------------------------------\n")
                    f.close()

loading the dataset
loaded the dataset
../results/csv_without_interms/DCT/3_Suture_90_30_1_report_DCT_fea.txt already exists
../results/csv_without_interms/DCT/4_Suture_90_30_1_report_DCT_fea.txt already exists
loading the dataset
loaded the dataset
../results/csv_without_interms/DCT/3_Knot_90_30_1_report_DCT_fea.txt already exists
../results/csv_without_interms/DCT/4_Knot_90_30_1_report_DCT_fea.txt already exists
loading the dataset
loaded the dataset
../results/csv_without_interms/DCT/3_Needle_90_30_1_report_DCT_fea.txt already exists
../results/csv_without_interms/DCT/4_Needle_90_30_1_report_DCT_fea.txt already exists
loading the dataset
loaded the dataset
../results/csv_without_interms/DCT/3_Suture_90_60_1_report_DCT_fea.txt already exists
../results/csv_without_interms/DCT/4_Suture_90_60_1_report_DCT_fea.txt already exists
loading the dataset
loaded the dataset
../results/csv_without_interms/DCT/3_Knot_90_60_1_report_DCT_fea.txt already exists
../results/csv_without_interms/DCT/4_

Accuracy: 0.2646048109965636
              precision    recall  f1-score   support

         0.0       0.48      0.35      0.41       389
         1.0       0.20      0.16      0.18        62
         2.0       0.03      0.06      0.04       131

    accuracy                           0.26       582
   macro avg       0.24      0.19      0.21       582
weighted avg       0.35      0.26      0.30       582

[[136  32 221]
 [ 30  10  22]
 [115   8   8]]
loading the dataset
loaded the dataset
User out:  1
Shuffle data
Accuracy: 0.782608695652174
              precision    recall  f1-score   support

         0.0       0.90      0.82      0.86       191
         1.0       0.33      0.04      0.07        25
         2.0       0.62      0.92      0.74        83

    accuracy                           0.78       299
   macro avg       0.62      0.59      0.56       299
weighted avg       0.78      0.78      0.76       299

[[157   2  32]
 [ 10   1  14]
 [  7   0  76]]
User out:  2
Shuffle dat

/home/gabor/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.538860103626943
              precision    recall  f1-score   support

         0.0       0.68      0.71      0.69       283
         1.0       0.00      0.00      0.00         0
         2.0       0.08      0.06      0.07       103

    accuracy                           0.54       386
   macro avg       0.25      0.26      0.25       386
weighted avg       0.52      0.54      0.53       386

[[202   8  73]
 [  0   0   0]
 [ 97   0   6]]
User out:  3
Shuffle data
Accuracy: 0.4642857142857143
              precision    recall  f1-score   support

         0.0       0.45      0.62      0.52       231
         1.0       0.08      0.03      0.04        70
         2.0       0.54      0.43      0.48       203

    accuracy                           0.46       504
   macro avg       0.36      0.36      0.35       504
weighted avg       0.44      0.46      0.44       504

[[144  19  68]
 [ 62   2   6]
 [112   3  88]]
User out:  4
Shuffle data
Accuracy: 0.3299798792756539
        

/home/gabor/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/home/gabor/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.4678362573099415
              precision    recall  f1-score   support

         0.0       0.51      0.79      0.62        91
         1.0       0.00      0.00      0.00         0
         2.0       0.33      0.10      0.15        80

    accuracy                           0.47       171
   macro avg       0.28      0.30      0.26       171
weighted avg       0.43      0.47      0.40       171

[[72  3 16]
 [ 0  0  0]
 [70  2  8]]
User out:  1
Shuffle data
Accuracy: 0.23655913978494625
              precision    recall  f1-score   support

         0.0       0.20      0.85      0.33        39
         1.0       0.00      0.00      0.00        28
         2.0       0.58      0.09      0.16       119

    accuracy                           0.24       186
   macro avg       0.26      0.31      0.16       186
weighted avg       0.41      0.24      0.17       186

[[ 33   0   6]
 [ 26   0   2]
 [103   5  11]]
User out:  2
Shuffle data


/home/gabor/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.5412371134020618
              precision    recall  f1-score   support

         0.0       0.68      0.73      0.70       142
         1.0       0.00      0.00      0.00         0
         2.0       0.06      0.04      0.05        52

    accuracy                           0.54       194
   macro avg       0.25      0.25      0.25       194
weighted avg       0.51      0.54      0.53       194

[[103   7  32]
 [  0   0   0]
 [ 49   1   2]]
User out:  3
Shuffle data
Accuracy: 0.42292490118577075
              precision    recall  f1-score   support

         0.0       0.42      0.63      0.51       116
         1.0       0.27      0.11      0.16        35
         2.0       0.45      0.29      0.36       102

    accuracy                           0.42       253
   macro avg       0.38      0.35      0.34       253
weighted avg       0.41      0.42      0.40       253

[[73 11 32]
 [27  4  4]
 [72  0 30]]
User out:  4
Shuffle data
Accuracy: 0.3815261044176707
              p

/home/gabor/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### ResNet Visualisation

In [1]:
import visualkeras
import scipy.io as sio
from keras import regularizers
from keras.layers import Input, Dense, GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv1D, BatchNormalization, Activation
from keras.layers import LSTM, add
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from sklearn import svm

input_root = '../data/augment_data_from_csvs/no_interm/'
win = 180
over = 30
skill = 'Knot'

dataset = sio.loadmat(input_root+skill+'_dataX_basic_'+str(win)+'_'+str(over)+'.mat')

features = dataset['dataX']
#print(features[0][1].shape, features[1][1].shape, features[2][1].shape)
#sizeD_ts = features[0][1][0][1][0][0][0][0][0].shape[0]
#sizeD_va = features[0][1][0][1][0][0][0][0][0].shape[1]
sizeD_ts = (((features[0,0:1])[0])[0,:])[0].shape[0]
sizeD_va = (((features[0,0:1])[0])[0,:])[0].shape[1]


#######---------Resnet------------
ip = Input(shape=(sizeD_ts, sizeD_va), name='main_input')

##### Block 1
x1 = Conv1D(64, 8, padding='same', kernel_initializer='he_uniform')(ip)
x1 = BatchNormalization()(x1)
x1 = Activation('relu')(x1)

x2 = Conv1D(64, 5, padding='same', kernel_initializer='he_uniform')(x1)
x2 = BatchNormalization()(x2)
x2 = Activation('relu')(x2)

x3 = Conv1D(64, 3, padding='same', kernel_initializer='he_uniform')(x2)
x3 = BatchNormalization()(x3)

# expand channels for the sum
sx2 = Conv1D(64, 1, padding='same', kernel_initializer='he_uniform')(ip)
sx2 = BatchNormalization()(sx2)

outputB1 = add([sx2, x3])
outputB1 = Activation('relu')(outputB1)

##### Block 2
x1 = Conv1D(64, 8, padding='same', kernel_initializer='he_uniform')(outputB1)
x1 = BatchNormalization()(x1)
x1 = Activation('relu')(x1)

x2 = Conv1D(64, 5, padding='same', kernel_initializer='he_uniform')(x1)
x2 = BatchNormalization()(x2)
x2 = Activation('relu')(x2)

x3 = Conv1D(64, 3, padding='same', kernel_initializer='he_uniform')(x2)
x3 = BatchNormalization()(x3)

# expand channels for the sum
sx2 = Conv1D(64, 1, padding='same', kernel_initializer='he_uniform')(outputB1)
sx2 = BatchNormalization()(sx2)

outputB2 = add([sx2, x3])
outputB2 = Activation('relu')(outputB2)

##### Block 3
x1 = Conv1D(64, 8, padding='same', kernel_initializer='he_uniform')(outputB2)
x1 = BatchNormalization()(x1)
x1 = Activation('relu')(x1)

x2 = Conv1D(64, 5, padding='same', kernel_initializer='he_uniform')(x1)
x2 = BatchNormalization()(x2)
x2 = Activation('relu')(x2)

x3 = Conv1D(64, 3, padding='same', kernel_initializer='he_uniform')(x2)
x3 = BatchNormalization()(x3)

#do not need to expand channels for the sum
sx2 = BatchNormalization()(outputB2)

outputB3 = add([sx2, x3])
outputB3 = Activation('relu')(outputB2)

#### End
fea = GlobalAveragePooling1D()(outputB3)
out = Dense(3, kernel_regularizer=regularizers.l2(0.001), activation='softmax')(fea)

model = Model(ip, out)

visualkeras.layered_view(model, to_file='resnet_model.png').show() # write and show

/home/gabor/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/gabor/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/gabor/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/gabor/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: Futu

Instructions for updating:
Colocations handled automatically by placer.


# ResNet

In [15]:
inputs = ['../data/augment_data_from_csvs/',
          '../data/augment_data_from_csvs/no_interm/',
]

outputs = ['../results/csv_based/',
           '../results/csv_without_interms/',
]

ind = 1

withoutInterms = (ind == 1)

output_root = outputs[ind]
input_root = inputs[ind]

if withoutInterms:
    print('Intermediate user data will be skipped')
else:
    print('All three user groups (Beginner, Intermediate, Expert) are processed')
    
groupRange = [0,2]
if not withoutInterms:
    groupRange = [0,1,2]
print(groupRange)

import os.path
from os import path

generateOnlyMissingResultFiles = True

All three user groups (Beginner, Intermediate, Expert) are processed
[0, 1, 2]


In [17]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 17 18:55:21 2019

@author: xngu0004
"""

import numpy as np
import random
import scipy.io as sio
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from keras import regularizers
from keras.layers import Input, Dense, GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv1D, BatchNormalization, Activation
from keras.layers import LSTM, add
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from sklearn import svm

#####################################################
"Select a configuration"

list_skill = ["Suture","Knot","Needle"]
win = 150#[90,120,150,180]
over = 60#[30,60]

####################################################
for skill in list_skill:
    for tim in range(1,6):        
        full_output_path = output_root + 'ResNet/' + str(tim)+'_'+skill+'_'+str(win)+'_'+str(over)+'_report_Resnet_fea.txt'
        if generateOnlyMissingResultFiles and path.exists(full_output_path):
            print(full_output_path + ' already exists')
            continue
        
        acc_1 = []
        accu = 0
        acc_1_fea = []
        accu_fea = 0
        # users in Group
        novice = ['B', 'G', 'H', 'I']
        intermediate = ['C', 'F']
        expert = ['D', 'E']

        " Load the data"
        " Reading the data "
        print('loading the dataset')
        dataset = sio.loadmat(input_root+skill+'_dataX_basic_'+str(win)+'_'+str(over)+'.mat')
        
        features = dataset['dataX']
        sizeD_ts = (((features[0,0:1])[0])[0,:])[0].shape[0]
        sizeD_va = (((features[0,0:1])[0])[0,:])[0].shape[1]
        print('loaded the dataset')
    
        ###############################################################################

        print('Pre-process data')
        for user in range(1,6):

            " Initialize data for concatenation "
            X_test1 = np.ones((1,sizeD_ts,sizeD_va))
            y_test1 = np.ones(1)
            X_train1 = np.ones((1,sizeD_ts,sizeD_va))
            y_train1 = np.ones(1)

            # Extract data for training and testing
            for j in groupRange:
                upper_range = (((features[0,j:j+1])[0])[0,:]).shape[0]
                for i in range(0, upper_range):
                    k = (((features[0,j:j+1])[0])[2,:])[i]
                    if (k == user): # take out the ith trial of each user for testing
                        temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                        temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                        X_test1 = np.concatenate((X_test1, temp_x1), axis=0) 
                        user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                        if(user_temp in novice):
                            y_test1 = np.concatenate((y_test1,[0]))
                        elif(user_temp in intermediate):
                            y_test1 = np.concatenate((y_test1,[1]))
                        elif(user_temp in expert):
                            y_test1 = np.concatenate((y_test1,[2]))
                    else:
                        temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                        temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                        X_train1 = np.concatenate((X_train1, temp_x1), axis=0)
                        user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                        if(user_temp in novice):
                            y_train1 = np.concatenate((y_train1,[0]))
                        elif(user_temp in intermediate):
                            y_train1 = np.concatenate((y_train1,[1]))
                        elif(user_temp in expert):
                            y_train1 = np.concatenate((y_train1,[2]))
        
            " Delete the first column "
            X_test1 = np.delete(X_test1,np.s_[0:1], axis=0)
            y_test1 = np.delete(y_test1,np.s_[0:1])
            X_train1 = np.delete(X_train1,np.s_[0:1], axis=0)
            y_train1 = np.delete(y_train1,np.s_[0:1])

            " Shuffle the training and testing data "
            X_test, y_test = shuffle(X_test1, y_test1, random_state = random.randint(10,50))
            X_train, y_train = shuffle(X_train1, y_train1, random_state = random.randint(10,50))
    
            classes = np.unique(y_train)
            le = LabelEncoder()
            y_ind = le.fit_transform(y_train.ravel())
            recip_freq = len(y_train) / (len(le.classes_) * np.bincount(y_ind).astype(np.float64))
            class_weight = recip_freq[le.transform(classes)]
    
            " One-hot coding"
            labels_train = to_categorical(y_train)
            labels_test = to_categorical(y_test)

            ###########################################################################
            print("Ready to train")
            
            #######---------Resnet------------
            ip = Input(shape=(sizeD_ts, sizeD_va), name='main_input')
    
            ##### Block 1
            x1 = Conv1D(64, 8, padding='same', kernel_initializer='he_uniform')(ip)
            x1 = BatchNormalization()(x1)
            x1 = Activation('relu')(x1)
            
            x2 = Conv1D(64, 5, padding='same', kernel_initializer='he_uniform')(x1)
            x2 = BatchNormalization()(x2)
            x2 = Activation('relu')(x2)
            
            x3 = Conv1D(64, 3, padding='same', kernel_initializer='he_uniform')(x2)
            x3 = BatchNormalization()(x3)

            # expand channels for the sum
            sx2 = Conv1D(64, 1, padding='same', kernel_initializer='he_uniform')(ip)
            sx2 = BatchNormalization()(sx2)
            
            outputB1 = add([sx2, x3])
            outputB1 = Activation('relu')(outputB1)
            
            ##### Block 2
            x1 = Conv1D(64, 8, padding='same', kernel_initializer='he_uniform')(outputB1)
            x1 = BatchNormalization()(x1)
            x1 = Activation('relu')(x1)
            
            x2 = Conv1D(64, 5, padding='same', kernel_initializer='he_uniform')(x1)
            x2 = BatchNormalization()(x2)
            x2 = Activation('relu')(x2)
            
            x3 = Conv1D(64, 3, padding='same', kernel_initializer='he_uniform')(x2)
            x3 = BatchNormalization()(x3)

            # expand channels for the sum
            sx2 = Conv1D(64, 1, padding='same', kernel_initializer='he_uniform')(outputB1)
            sx2 = BatchNormalization()(sx2)
            
            outputB2 = add([sx2, x3])
            outputB2 = Activation('relu')(outputB2)
            
            ##### Block 3
            x1 = Conv1D(64, 8, padding='same', kernel_initializer='he_uniform')(outputB2)
            x1 = BatchNormalization()(x1)
            x1 = Activation('relu')(x1)
            
            x2 = Conv1D(64, 5, padding='same', kernel_initializer='he_uniform')(x1)
            x2 = BatchNormalization()(x2)
            x2 = Activation('relu')(x2)
            
            x3 = Conv1D(64, 3, padding='same', kernel_initializer='he_uniform')(x2)
            x3 = BatchNormalization()(x3)

            #do not need to expand channels for the sum
            sx2 = BatchNormalization()(outputB2)
            
            outputB3 = add([sx2, x3])
            outputB3 = Activation('relu')(outputB2)
            
            #### End
            fea = GlobalAveragePooling1D()(outputB3)
            out = Dense(3, kernel_regularizer=regularizers.l2(0.001), activation='softmax')(fea)
            
            model = Model(ip, out)
######---------------------
            
            epochs_s = 100
            batch_s = 16
            learning_rate = 1e-3   
            weight_fn = "./weights/weights_fea_Resnet_t"+str(user)+".h5"
            model_checkpoint = ModelCheckpoint(weight_fn, verbose=1, mode='max', monitor='val_acc', save_best_only=True, save_weights_only=True)
            stop = EarlyStopping(monitor='val_loss', patience=15)
            callback_list = [model_checkpoint, stop]
            optm = Adam(lr=learning_rate)

            model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
            model.fit(X_train, labels_train, batch_size=batch_s, epochs=epochs_s, callbacks = callback_list, class_weight=class_weight, verbose=2, validation_data=(X_test, labels_test))

            model.load_weights(weight_fn)
            prediction = model.predict(X_test);
            y_pred = np.argmax(prediction, axis=1)
    
            cr = classification_report(y_test, y_pred)
            cm = confusion_matrix(y_test, y_pred)
    
            print("-------------------- Full NN -----------------------")
            print("Skill: ", skill)
            print("Trial out:", user)
            acc = np.sum(y_pred == y_test)/y_pred.shape[0]
            acc_1.append(acc)
            print('Accuracy: ' + str(acc))
            print(cr)
            print(cm)
            f = open(full_output_path, 'a+')
            f.write("-------------------- Full NN -----------------------")
            f.write('---------' + skill + " Win: " + str(win) + " Shift: " + str(over) + " Trial: " + str(user) + '---------\n\n')
            f.write('Accuracy: ' + str(acc))
            f.write('\n\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n\n'.format(cr, cm))
            
            ##############################################
            moFea = Model(ip, fea)
            X_train_fea = moFea.predict(X_train)
            X_test_fea = moFea.predict(X_test)
            svm1 = svm.SVC(kernel = 'linear')
            svm1.fit(X_train_fea, y_train)
            prediction_fea = svm1.predict(X_test_fea)
            y_pred_fea = prediction_fea
            cr_fea = classification_report(y_test, y_pred_fea)
            cm_fea = confusion_matrix(y_test, y_pred_fea)
    
            acc_fea = np.sum(y_pred_fea == y_test)/y_pred_fea.shape[0]
            acc_1_fea.append(acc_fea)
            print('Accuracy: ' + str(acc_fea))
            print(cr_fea)
            print(cm_fea)
            f.write("-------------------- NN Fea -----------------------")
            f.write('---------' + skill + " Win: " + str(win) + " Shift: " + str(over) + " Trial: " + str(user) + '---------\n\n')
            f.write('Accuracy: ' + str(acc_fea))
            f.write('\n\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n\n'.format(cr_fea, cm_fea))
        for i in range(0,5):
            accu += acc_1[i]
            accu_fea += acc_1_fea[i]
        accu = accu/5
        accu_fea = accu_fea/5
        f.write("\n-------------------------------------------\n")
        f.write("Acc_full = " + str(accu))
        f.write("\nAcc_fea = " + str(accu_fea))
        f.write("\n-------------------------------------------\n")
        f.close()

../results/csv_based/ResNet/1_Suture_150_60_report_Resnet_fea.txt already exists
../results/csv_based/ResNet/2_Suture_150_60_report_Resnet_fea.txt already exists
../results/csv_based/ResNet/3_Suture_150_60_report_Resnet_fea.txt already exists
../results/csv_based/ResNet/4_Suture_150_60_report_Resnet_fea.txt already exists
../results/csv_based/ResNet/5_Suture_150_60_report_Resnet_fea.txt already exists
../results/csv_based/ResNet/1_Knot_150_60_report_Resnet_fea.txt already exists
../results/csv_based/ResNet/2_Knot_150_60_report_Resnet_fea.txt already exists
../results/csv_based/ResNet/3_Knot_150_60_report_Resnet_fea.txt already exists
../results/csv_based/ResNet/4_Knot_150_60_report_Resnet_fea.txt already exists
../results/csv_based/ResNet/5_Knot_150_60_report_Resnet_fea.txt already exists
../results/csv_based/ResNet/1_Needle_150_60_report_Resnet_fea.txt already exists
../results/csv_based/ResNet/2_Needle_150_60_report_Resnet_fea.txt already exists
../results/csv_based/ResNet/3_Needle_1

# LSTM

In [12]:
inputs = ['../data/augment_data_from_csvs/',
          '../data/augment_data_from_csvs/no_interm/',
]

outputs = ['../results/csv_based/',
           '../results/csv_without_interms/',
]

ind = 1

withoutInterms = (ind == 1)

output_root = outputs[ind]
input_root = inputs[ind]

if withoutInterms:
    print('Intermediate user data will be skipped')
else:
    print('All three user groups (Beginner, Intermediate, Expert) are processed')
    
groupRange = [0,2]
if not withoutInterms:
    groupRange = [0,1,2]
print(groupRange)

import os.path
from os import path

generateOnlyMissingResultFiles = True

All three user groups (Beginner, Intermediate, Expert) are processed
[0, 1, 2]


In [14]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 22 22:04:11 2019

@author: xngu0004
"""

import numpy as np
import random
import scipy.io as sio
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from keras import regularizers
from keras.layers import Input, Dense, GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv1D, BatchNormalization, Activation
from keras.layers import LSTM
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from sklearn import svm

#####################################################
"Select a configuration"

list_skill = ["Suture","Knot","Needle"]
win = 180#[90,120,150,180]
over = 30#[30,60]

####################################################
for skill in list_skill:
    for tim in range(1,6):
        full_output_path = output_root + 'LSTM/' + str(tim)+'_'+skill+'_'+str(win)+'_'+str(over)+'_report_LSTM_fea.txt'
        if generateOnlyMissingResultFiles and path.exists(full_output_path):
            print(full_output_path + ' already exists')
            continue
        acc_1 = []
        accu = 0
        acc_1_fea = []
        accu_fea = 0
        # users in Group
        novice = ['B', 'G', 'H', 'I']
        intermediate = ['C', 'F']
        expert = ['D', 'E']

        " Load the data"
        " Reading the data "
        print('loading the dataset')
        dataset = sio.loadmat(input_root + skill+'_dataX_basic_'+str(win)+'_'+str(over)+'.mat')

        features = dataset['dataX']
        sizeD_ts = (((features[0,0:1])[0])[0,:])[0].shape[0]
        sizeD_va = (((features[0,0:1])[0])[0,:])[0].shape[1]
        print('loaded the dataset')
    
        ###############################################################################

        print('Pre-process data')
        for user in range(1,6):

            " Initialize data for concatenation "
            X_test1 = np.ones((1,sizeD_ts,sizeD_va))
            y_test1 = np.ones(1)
            X_train1 = np.ones((1,sizeD_ts,sizeD_va))
            y_train1 = np.ones(1)

            # Extract data for training and testing
            for j in groupRange:
                for i in range(0, (((features[0,j:j+1])[0])[0,:]).shape[0]):
                    k = (((features[0,j:j+1])[0])[2,:])[i]
                    if (k == user): # take out the ith trial of each user for testing
                        temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                        temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                        X_test1 = np.concatenate((X_test1, temp_x1), axis=0) 
                        user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                        if(user_temp in novice):
                            y_test1 = np.concatenate((y_test1,[0]))
                        elif(user_temp in intermediate):
                            y_test1 = np.concatenate((y_test1,[1]))
                        elif(user_temp in expert):
                            y_test1 = np.concatenate((y_test1,[2]))
                    else:
                        temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                        temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                        X_train1 = np.concatenate((X_train1, temp_x1), axis=0)
                        user_temp = (((features[0,j:j+1])[0])[1,:])[i]
        
                        if(user_temp in novice):
                            y_train1 = np.concatenate((y_train1,[0]))
                        elif(user_temp in intermediate):
                            y_train1 = np.concatenate((y_train1,[1]))
                        elif(user_temp in expert):
                            y_train1 = np.concatenate((y_train1,[2]))
        
            " Delete the first column "
            X_test1 = np.delete(X_test1,np.s_[0:1], axis=0)
            y_test1 = np.delete(y_test1,np.s_[0:1])
            X_train1 = np.delete(X_train1,np.s_[0:1], axis=0)
            y_train1 = np.delete(y_train1,np.s_[0:1])

            " Shuffle the training and testing data "
            X_test, y_test = shuffle(X_test1, y_test1, random_state = random.randint(10,50))
            X_train, y_train = shuffle(X_train1, y_train1, random_state = random.randint(10,50))
    
            classes = np.unique(y_train)
            le = LabelEncoder()
            y_ind = le.fit_transform(y_train.ravel())
            recip_freq = len(y_train) / (len(le.classes_) * np.bincount(y_ind).astype(np.float64))
            class_weight = recip_freq[le.transform(classes)]
    
            " One-hot coding"
            labels_train = to_categorical(y_train)
            labels_test = to_categorical(y_test)

            ###########################################################################
            print("Ready to train")

            #######----------LSTM-----------START----######
            ip = Input(shape=(sizeD_ts, sizeD_va), name='main_input')
    
            x = LSTM(32, return_sequences=True)(ip)
            fea = LSTM(32)(x)

            out = Dense(3, kernel_regularizer=regularizers.l2(0.001), activation='softmax')(fea)

            model = Model(ip, out)
            #######----------LSTM-----------END-----######
            
            epochs_s = 50
            batch_s = 16
            learning_rate = 1e-3   
            weight_fn = "./weights/weights_fea_LSTM"+str(user)+".h5"
            model_checkpoint = ModelCheckpoint(weight_fn, verbose=1, mode='max', monitor='val_acc', save_best_only=True, save_weights_only=True)
            stop = EarlyStopping(monitor='val_loss', patience=15)
            callback_list = [model_checkpoint, stop]
            optm = Adam(lr=learning_rate)

            model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
            model.fit(X_train, labels_train, batch_size=batch_s, epochs=epochs_s, callbacks = callback_list, class_weight=class_weight, verbose=2, validation_data=(X_test, labels_test))

            model.load_weights(weight_fn)
            prediction = model.predict(X_test);
            y_pred = np.argmax(prediction, axis=1)
    
            cr = classification_report(y_test, y_pred)
            cm = confusion_matrix(y_test, y_pred)
    
            print("-------------------- Full NN -----------------------")
            print("Skill: ", skill)
            print("Trial out:", user)
            acc = np.sum(y_pred == y_test)/y_pred.shape[0]
            acc_1.append(acc)
            print('Accuracy: ' + str(acc))
            print(cr)
            print(cm)
            f = open(full_output_path, 'a+')
            f.write("-------------------- Full NN -----------------------")
            f.write('---------' + skill + " Win: " + str(win) + " Shift: " + str(over) + " Trial: " + str(user) + '---------\n\n')
            f.write('Accuracy: ' + str(acc))
            f.write('\n\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n\n'.format(cr, cm))
            
            ##############################################
            moFea = Model(ip, fea)
            X_train_fea = moFea.predict(X_train)
            X_test_fea = moFea.predict(X_test)
            svm1 = svm.SVC(kernel = 'linear')
            svm1.fit(X_train_fea, y_train)
            prediction_fea = svm1.predict(X_test_fea)
            y_pred_fea = prediction_fea
            cr_fea = classification_report(y_test, y_pred_fea)
            cm_fea = confusion_matrix(y_test, y_pred_fea)
    
            acc_fea = np.sum(y_pred_fea == y_test)/y_pred_fea.shape[0]
            acc_1_fea.append(acc_fea)
            print('Accuracy: ' + str(acc_fea))
            print(cr_fea)
            print(cm_fea)
            f.write("-------------------- NN Fea -----------------------")
            f.write('---------' + skill + " Win: " + str(win) + " Shift: " + str(over) + " Trial: " + str(user) + '---------\n\n')
            f.write('Accuracy: ' + str(acc_fea))
            f.write('\n\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n\n'.format(cr_fea, cm_fea))
        for i in range(0,5):
            accu += acc_1[i]
            accu_fea += acc_1_fea[i]
        accu = accu/5
        accu_fea = accu_fea/5
        f.write("\n-------------------------------------------\n")
        f.write("Acc_full = " + str(accu))
        f.write("\nAcc_fea = " + str(accu_fea))
        f.write("\n-------------------------------------------\n")
        f.close()

../results/csv_based/LSTM/1_Suture_180_30_report_LSTM_fea.txt already exists
../results/csv_based/LSTM/2_Suture_180_30_report_LSTM_fea.txt already exists
../results/csv_based/LSTM/3_Suture_180_30_report_LSTM_fea.txt already exists
../results/csv_based/LSTM/4_Suture_180_30_report_LSTM_fea.txt already exists
../results/csv_based/LSTM/5_Suture_180_30_report_LSTM_fea.txt already exists
../results/csv_based/LSTM/1_Knot_180_30_report_LSTM_fea.txt already exists
../results/csv_based/LSTM/2_Knot_180_30_report_LSTM_fea.txt already exists
../results/csv_based/LSTM/3_Knot_180_30_report_LSTM_fea.txt already exists
../results/csv_based/LSTM/4_Knot_180_30_report_LSTM_fea.txt already exists
../results/csv_based/LSTM/5_Knot_180_30_report_LSTM_fea.txt already exists
../results/csv_based/LSTM/1_Needle_180_30_report_LSTM_fea.txt already exists
../results/csv_based/LSTM/2_Needle_180_30_report_LSTM_fea.txt already exists
../results/csv_based/LSTM/3_Needle_180_30_report_LSTM_fea.txt already exists
../result

# CNN

In [9]:
inputs = ['../data/augment_data_from_csvs/',
          '../data/augment_data_from_csvs/no_interm/',
]

outputs = ['../results/csv_based/',
           '../results/csv_without_interms/',
]

ind = 1

withoutInterms = (ind == 1)

output_root = outputs[ind]
input_root = inputs[ind]

if withoutInterms:
    print('Intermediate user data will be skipped')
else:
    print('All three user groups (Beginner, Intermediate, Expert) are processed')
    
groupRange = [0,2]
if not withoutInterms:
    groupRange = [0,1,2]
print(groupRange)

import os.path
from os import path

generateOnlyMissingResultFiles = True

All three user groups (Beginner, Intermediate, Expert) are processed
[0, 1, 2]


In [11]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 17 18:55:38 2019

@author: xngu0004
"""

import numpy as np
import random
import scipy.io as sio
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from keras import regularizers
from keras.layers import Input, Dense, GlobalAveragePooling1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv1D, BatchNormalization, Activation
from keras.layers import LSTM
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from sklearn import svm
import matplotlib.pyplot as plt

#####################################################
"Select a configuration"

list_skill = [ "Knot", "Needle", "Suture"]
win = 150#[90, 120, 150, 180]
over = 60#[30, 60]

####################################################
for skill in list_skill:
    for tim in range(1,6):
        full_output_path = output_root + 'CNN/' + str(tim)+'_'+skill+'_'+str(win)+'_'+str(over)+'_report_CNN_fea.txt'
        if generateOnlyMissingResultFiles and path.exists(full_output_path):
            print(full_output_path + ' already exists')
            continue
        acc_1 = []
        accu = 0
        acc_1_fea = []
        accu_fea = 0
        # users in Group
        novice = ['B', 'G', 'H', 'I']
        intermediate = ['C', 'F']
        expert = ['D', 'E']

        " Load the data"
        " Reading the data "
        print('loading the dataset')
        dataset = sio.loadmat(input_root+skill+'_dataX_basic_'+str(win)+'_'+str(over)+'.mat')

        features = dataset['dataX']
        sizeD_ts = (((features[0,0:1])[0])[0,:])[0].shape[0]
        sizeD_va = (((features[0,0:1])[0])[0,:])[0].shape[1]
        print('loaded the dataset')
    
        ###############################################################################

        print('Pre-process data')
        for user in range(1,6):

            " Initialize data for concatenation "
            X_test1 = np.ones((1,sizeD_ts,sizeD_va))
            y_test1 = np.ones(1)
            X_train1 = np.ones((1,sizeD_ts,sizeD_va))
            y_train1 = np.ones(1)

            # Extract data for training and testing
            for j in groupRange:
                for i in range(0, (((features[0,j:j+1])[0])[0,:]).shape[0]):
                    k = (((features[0,j:j+1])[0])[2,:])[i]
                    if (k == user): # take out the ith trial of each user for testing
                        temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                        temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                        X_test1 = np.concatenate((X_test1, temp_x1), axis=0) 
                        user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                        
                        if(user_temp in novice):
                            y_test1 = np.concatenate((y_test1,[0]))
                        elif(user_temp in intermediate):
                            y_test1 = np.concatenate((y_test1,[1]))
                        elif(user_temp in expert):
                            y_test1 = np.concatenate((y_test1,[2]))
                    else:
                        temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                        temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                        X_train1 = np.concatenate((X_train1, temp_x1), axis=0)
                        user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                        
                        if(user_temp in novice):
                            y_train1 = np.concatenate((y_train1,[0]))
                        elif(user_temp in intermediate):
                            y_train1 = np.concatenate((y_train1,[1]))
                        elif(user_temp in expert):
                            y_train1 = np.concatenate((y_train1,[2]))
            
            " Delete the first column "
            X_test1 = np.delete(X_test1,np.s_[0:1], axis=0)
            y_test1 = np.delete(y_test1,np.s_[0:1])
            X_train1 = np.delete(X_train1,np.s_[0:1], axis=0)
            y_train1 = np.delete(y_train1,np.s_[0:1])

            " Shuffle the training and testing data "
            X_test, y_test = shuffle(X_test1, y_test1, random_state = random.randint(10,50))
            X_train, y_train = shuffle(X_train1, y_train1, random_state = random.randint(10,50))
    
            classes = np.unique(y_train)
            le = LabelEncoder()
            y_ind = le.fit_transform(y_train.ravel())
            recip_freq = len(y_train) / (len(le.classes_) * np.bincount(y_ind).astype(np.float64))
            class_weight = recip_freq[le.transform(classes)]
    
            " One-hot coding"
            labels_train = to_categorical(y_train)
            labels_test = to_categorical(y_test)

            ###########################################################################
            print("Ready to train")
            #model = model_arch2(withOP=True)
            
            #######-------- CNN -------------START----#####
            ip = Input(shape=(sizeD_ts, sizeD_va), name='main_input')
    
            x = Conv1D(16, 7, padding='same', kernel_initializer='he_uniform')(ip)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv1D(32, 5, padding='same', kernel_initializer='he_uniform')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            fea = GlobalAveragePooling1D()(x)

            out = Dense(3, kernel_regularizer=regularizers.l2(0.001), activation='softmax')(fea)

            model = Model(ip, out)
            #######-------- CNN -------------END----######
            
            epochs_s = 100
            batch_s = 16
            learning_rate = 1e-3   
            weight_fn = "./weights/weights_fea_CNNi"+str(user)+".h5"
            model_checkpoint = ModelCheckpoint(weight_fn, verbose=1, mode='max', monitor='val_acc', save_best_only=True, save_weights_only=True)
            stop = EarlyStopping(monitor='val_loss', patience=15)
            callback_list = [model_checkpoint, stop]
            optm = Adam(lr=learning_rate)

            model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
            history = model.fit(X_train, labels_train, batch_size=batch_s, epochs=epochs_s, callbacks = callback_list, class_weight=class_weight, verbose=2, validation_data=(X_test, labels_test))
            
#            plt.plot(history.history['acc'])
#            plt.plot(history.history['val_acc'])
#            plt.title('model accuracy')
#            plt.ylabel('accuracy')
#            plt.xlabel('epoch')
#            plt.legend(['train', 'test'], loc='upper left')
#            plt.savefig(str(tim)+'_'+skill+'_'+str(win)+'_'+str(over)+"_user_"+str(user)+'acc.pdf')
#            plt.close()
#            
#            plt.plot(history.history['loss'])
#            plt.plot(history.history['val_loss'])
#            plt.title('model loss')
#            plt.ylabel('loss')
#            plt.xlabel('epoch')
#            plt.legend(['train', 'test'], loc='upper left')
#            plt.savefig(str(tim)+'_'+skill+'_'+str(win)+'_'+str(over)+"_user_"+str(user)+'lost.pdf')
#            plt.close()
            
            model.load_weights(weight_fn)
            prediction = model.predict(X_test);
            y_pred = np.argmax(prediction, axis=1)
    
            cr = classification_report(y_test, y_pred)
            cm = confusion_matrix(y_test, y_pred)
    
            print("-------------------- Full NN -----------------------")
            print("Skill: ", skill)
            print("Trial out:", user)
            acc = np.sum(y_pred == y_test)/y_pred.shape[0]
            acc_1.append(acc)
            print('Accuracy: ' + str(acc))
            print(cr)
            print(cm)
            f = open(full_output_path, 'a+')
            f.write("-------------------- Full NN -----------------------")
            f.write('---------' + skill + " Win: " + str(win) + " Shift: " + str(over) + " Trial: " + str(user) + '---------\n\n')
            f.write('Accuracy: ' + str(acc))
            f.write('\n\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n\n'.format(cr, cm))
            
            ##############################################
            moFea = Model(ip, fea)
            X_train_fea = moFea.predict(X_train)
            X_test_fea = moFea.predict(X_test)
            svm1 = svm.SVC(kernel = 'linear')
            svm1.fit(X_train_fea, y_train)
            prediction_fea = svm1.predict(X_test_fea)
            
            y_pred_fea = prediction_fea
            cr_fea = classification_report(y_test, y_pred_fea)
            cm_fea = confusion_matrix(y_test, y_pred_fea)
    
            acc_fea = np.sum(y_pred_fea == y_test)/y_pred_fea.shape[0]
            acc_1_fea.append(acc_fea)
            print('Accuracy: ' + str(acc_fea))
            print(cr_fea)
            print(cm_fea)
            f.write("-------------------- NN Fea -----------------------")
            f.write('---------' + skill + " Win: " + str(win) + " Shift: " + str(over) + " Trial: " + str(user) + '---------\n\n')
            f.write('Accuracy: ' + str(acc_fea))
            f.write('\n\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n\n'.format(cr_fea, cm_fea))
        for i in range(0,5):
            accu += acc_1[i]
            accu_fea += acc_1_fea[i]
        accu = accu/5
        accu_fea = accu_fea/5
        f.write("\n-------------------------------------------\n")
        f.write("Acc_full = " + str(accu))
        f.write("\nAcc_fea = " + str(accu_fea))
        f.write("\n-------------------------------------------\n")
        f.close()

../results/csv_based/CNN/1_Knot_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/2_Knot_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/3_Knot_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/4_Knot_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/5_Knot_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/1_Needle_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/2_Needle_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/3_Needle_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/4_Needle_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/5_Needle_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/1_Suture_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/2_Suture_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/3_Suture_150_60_report_CNN_fea.txt already exists
../results/csv_based/CNN/4_Suture_1

# CNN + LSTM

In [6]:
inputs = ['../data/augment_data_from_csvs/',
          '../data/augment_data_from_csvs/no_interm/',
]

outputs = ['../results/csv_based/',
           '../results/csv_without_interms/',
]

ind = 1

withoutInterms = (ind == 1)

output_root = outputs[ind]
input_root = inputs[ind]

if withoutInterms:
    print('Intermediate user data will be skipped')
else:
    print('All three user groups (Beginner, Intermediate, Expert) are processed')
    
groupRange = [0,2]
if not withoutInterms:
    groupRange = [0,1,2]
print(groupRange)

import os.path
from os import path

generateOnlyMissingResultFiles = True

All three user groups (Beginner, Intermediate, Expert) are processed
[0, 1, 2]


In [8]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 17 13:21:11 2019

@author: xngu0004
"""

import numpy as np
import random
import scipy.io as sio
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from keras import regularizers
from keras.layers import Input, Dense, GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv1D, BatchNormalization, Activation
from keras.layers import LSTM
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from sklearn import svm

#####################################################
"Select a configuration"

list_skill = ["Suture","Knot","Needle"]
win = 180#[90,120,150,180]
over = 30#[30,60]

# users in Group
novice = ['B', 'G', 'H', 'I']
intermediate = ['C', 'F']
expert = ['D', 'E']
####################################################
for skill in list_skill:
    " Load the data"
    print('loading the dataset')
    dataset = sio.loadmat(input_root+skill+'_dataX_basic_'+str(win)+'_'+str(over)+'.mat')
    features = dataset['dataX']
    sizeD_ts = (((features[0,0:1])[0])[0,:])[0].shape[0]
    sizeD_va = (((features[0,0:1])[0])[0,:])[0].shape[1]
    print('loaded the dataset')
    
    for tim in range(1,6):
        full_output_path = output_root + 'CNN+LSTM/' + str(tim)+'_'+skill+'_'+str(win)+'_'+str(over)+'_report_CNNLSTM_fea.txt'
        if generateOnlyMissingResultFiles and path.exists(full_output_path):
            print(full_output_path + ' already exists')
            continue
        acc_1 = []
        accu = 0
        acc_1_fea = []
        accu_fea = 0

        ##################################################
        print('Pre-process data')
        for user in range(1,6):

            " Initialize data for concatenation "
            X_test1 = np.ones((1,sizeD_ts,sizeD_va))
            y_test1 = np.ones(1)
            X_train1 = np.ones((1,sizeD_ts,sizeD_va))
            y_train1 = np.ones(1)

            # Extract data for training and testing
            for j in groupRange:
                for i in range(0, (((features[0,j:j+1])[0])[0,:]).shape[0]):
                    k = (((features[0,j:j+1])[0])[2,:])[i]
                    if (k == user): # take out the ith trial of each user for testing
                        temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                        temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                        X_test1 = np.concatenate((X_test1, temp_x1), axis=0) 
                        user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                        if(user_temp in novice):
                            y_test1 = np.concatenate((y_test1,[0]))
                        elif(user_temp in intermediate):
                            y_test1 = np.concatenate((y_test1,[1]))
                        elif(user_temp in expert):
                            y_test1 = np.concatenate((y_test1,[2]))
                    else:
                        temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                        temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                        X_train1 = np.concatenate((X_train1, temp_x1), axis=0)
                        user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                        
                        if(user_temp in novice):
                            y_train1 = np.concatenate((y_train1,[0]))
                        elif(user_temp in intermediate):
                            y_train1 = np.concatenate((y_train1,[1]))
                        elif(user_temp in expert):
                            y_train1 = np.concatenate((y_train1,[2]))
        
            " Delete the first column "
            X_test1 = np.delete(X_test1,np.s_[0:1], axis=0)
            y_test1 = np.delete(y_test1,np.s_[0:1])
            X_train1 = np.delete(X_train1,np.s_[0:1], axis=0)
            y_train1 = np.delete(y_train1,np.s_[0:1])
            print("Data shuffle")
            " Shuffle the training and testing data "
            X_test, y_test = shuffle(X_test1, y_test1, random_state = random.randint(10,50))
            X_train, y_train = shuffle(X_train1, y_train1, random_state = random.randint(10,50))
    
            classes = np.unique(y_train)
            le = LabelEncoder()
            y_ind = le.fit_transform(y_train.ravel())
            recip_freq = len(y_train) / (len(le.classes_) * np.bincount(y_ind).astype(np.float64))
            class_weight = recip_freq[le.transform(classes)]
    
            " One-hot coding"
            labels_train = to_categorical(y_train)
            labels_test = to_categorical(y_test)

            ###########################################################################
            print("Ready to train")
            #######---------CNN+LSTM------------START---#####
            ip = Input(shape=(sizeD_ts, sizeD_va), name='main_input')
    
            x = Conv1D(16, 7, padding='same', kernel_initializer='he_uniform')(ip)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv1D(32, 5, padding='same', kernel_initializer='he_uniform')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
    
            x = LSTM(32, return_sequences=True)(x)
            fea = LSTM(32)(x)

            out = Dense(3, kernel_regularizer=regularizers.l2(0.001), activation='softmax')(fea)
            
            model = Model(ip, out)
            
            #######------CNN+LSTM----------END----#####
            
            epochs_s = 100
            batch_s = 16
            learning_rate = 1e-3   
            weight_fn = "./weights/weights_fea_CNNLSTM_"+str(user)+".h5"
            model_checkpoint = ModelCheckpoint(weight_fn, verbose=1, mode='max', monitor='val_acc', save_best_only=True, save_weights_only=True)
            stop = EarlyStopping(monitor='val_loss', patience=15)
            callback_list = [model_checkpoint, stop]
            optm = Adam(lr=learning_rate)

            model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
            model.fit(X_train, labels_train, batch_size=batch_s, epochs=epochs_s, callbacks = callback_list, class_weight=class_weight, verbose=2, validation_data=(X_test, labels_test))

            model.load_weights(weight_fn)
            prediction = model.predict(X_test);
            y_pred = np.argmax(prediction, axis=1)
    
            cr = classification_report(y_test, y_pred)
            cm = confusion_matrix(y_test, y_pred)
    
            print("-------------------- Full NN -----------------------")
            print("Skill: ", skill)
            print("Trial out:", user)
            acc = np.sum(y_pred == y_test)/y_pred.shape[0]
            acc_1.append(acc)
            print('Accuracy: ' + str(acc))
            print(cr)
            print(cm)
            f = open(full_output_path, 'a+')
            f.write("-------------------- Full NN -----------------------")
            f.write('---------' + skill + " Win: " + str(win) + " Shift: " + str(over) + " Trial: " + str(user) + '---------\n\n')
            f.write('Accuracy: ' + str(acc))
            f.write('\n\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n\n'.format(cr, cm))
            
            ##############################################
            moFea = Model(ip, fea)
            X_train_fea = moFea.predict(X_train)
            X_test_fea = moFea.predict(X_test)
            svm1 = svm.SVC(kernel = 'linear')
            svm1.fit(X_train_fea, y_train)
            prediction_fea = svm1.predict(X_test_fea)
            y_pred_fea = prediction_fea
            cr_fea = classification_report(y_test, y_pred_fea)
            cm_fea = confusion_matrix(y_test, y_pred_fea)
    
            acc_fea = np.sum(y_pred_fea == y_test)/y_pred_fea.shape[0]
            acc_1_fea.append(acc_fea)
            print('Accuracy: ' + str(acc_fea))
            print(cr_fea)
            print(cm_fea)
            f.write("-------------------- NN Fea -----------------------")
            f.write('---------' + skill + " Win: " + str(win) + " Shift: " + str(over) + " Trial: " + str(user) + '---------\n\n')
            f.write('Accuracy: ' + str(acc_fea))
            f.write('\n\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n\n'.format(cr_fea, cm_fea))
        for i in range(0,5):
            accu += acc_1[i]
            accu_fea += acc_1_fea[i]
        accu = accu/5
        accu_fea = accu_fea/5
        f.write("\n-------------------------------------------\n")
        f.write("Acc_full = " + str(accu))
        f.write("\nAcc_fea = " + str(accu_fea))
        f.write("\n-------------------------------------------\n")
        f.close()

loading the dataset
loaded the dataset
../results/csv_based/CNN+LSTM/1_Suture_180_30_report_CNNLSTM_fea.txt already exists
../results/csv_based/CNN+LSTM/2_Suture_180_30_report_CNNLSTM_fea.txt already exists
../results/csv_based/CNN+LSTM/3_Suture_180_30_report_CNNLSTM_fea.txt already exists
../results/csv_based/CNN+LSTM/4_Suture_180_30_report_CNNLSTM_fea.txt already exists
../results/csv_based/CNN+LSTM/5_Suture_180_30_report_CNNLSTM_fea.txt already exists
loading the dataset
loaded the dataset
../results/csv_based/CNN+LSTM/1_Knot_180_30_report_CNNLSTM_fea.txt already exists
../results/csv_based/CNN+LSTM/2_Knot_180_30_report_CNNLSTM_fea.txt already exists
../results/csv_based/CNN+LSTM/3_Knot_180_30_report_CNNLSTM_fea.txt already exists
../results/csv_based/CNN+LSTM/4_Knot_180_30_report_CNNLSTM_fea.txt already exists
../results/csv_based/CNN+LSTM/5_Knot_180_30_report_CNNLSTM_fea.txt already exists
loading the dataset
loaded the dataset
../results/csv_based/CNN+LSTM/1_Needle_180_30_report_

# ConvAuto

In [4]:
inputs = ['../data/augment_data_from_csvs/',
          '../data/augment_data_from_csvs/no_interm/',
]

outputs = ['../results/csv_based/',
           '../results/csv_without_interms/',
]

ind = 1

withoutInterms = (ind == 1)

output_root = outputs[ind]
input_root = inputs[ind]

if withoutInterms:
    print('Intermediate user data will be skipped')
else:
    print('All three user groups (Beginner, Intermediate, Expert) are processed')
    
groupRange = [0,2]
if not withoutInterms:
    groupRange = [0,1,2]
print(groupRange)

import os.path
from os import path

generateOnlyMissingResultFiles = True

All three user groups (Beginner, Intermediate, Expert) are processed
[0, 1, 2]


In [5]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  7 16:36:07 2019

@author: xngu0004
"""

import numpy as np
import random
import scipy.io as sio
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from keras import regularizers
from keras.layers import Input, Dense, GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv1D, BatchNormalization, Activation, Flatten
from keras.layers import LSTM, Dense, MaxPooling1D, UpSampling1D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

#####################################################
"Select a configuration"

list_skill = ["Suture", "Knot", "Needle"]
win = 120#90, 120, 150, 180]
over = 60#[30, 60]

####################################################
for skill in list_skill:
    for tim in range(1,6):
        full_output_path = output_root + 'convAuto/' + str(tim)+'_'+skill+'_'+str(win)+'_'+str(over)+'_report_AutoCNN_fea.txt'
        if generateOnlyMissingResultFiles and path.exists(full_output_path):
            print(full_output_path + ' already exists')
            continue
        acc_1 = []
        accu = 0
        acc_1_fea = []
        accu_fea = 0
        # users in Group
        novice = ['B', 'G', 'H', 'I']
        intermediate = ['C', 'F']
        expert = ['D', 'E']

        " Load the data"
        " Reading the data "
        print('loading the dataset')
        dataset = sio.loadmat(input_root+skill+'_dataX_basic_'+str(win)+'_'+str(over)+'.mat')

        features = dataset['dataX']
        sizeD_ts = (((features[0,0:1])[0])[0,:])[0].shape[0]
        sizeD_va = (((features[0,0:1])[0])[0,:])[0].shape[1]
        print('loaded the dataset')
    
        ###############################################################################

        print('Pre-process data')
        for user in range(1,6):

            " Initialize data for concatenation "
            X_test1 = np.ones((1,sizeD_ts,sizeD_va))
            y_test1 = np.ones(1)
            X_train1 = np.ones((1,sizeD_ts,sizeD_va))
            y_train1 = np.ones(1)

            # Extract data for training and testing
            for j in groupRange:
                upper_range = (((features[0,j:j+1])[0])[0,:]).shape[0]
                for i in range(0, upper_range):
                    k = (((features[0,j:j+1])[0])[2,:])[i]
                    if (k == user): # take out the ith trial of each user for testing
                        temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                        temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                        X_test1 = np.concatenate((X_test1, temp_x1), axis=0) 
                        user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                        if(user_temp in novice):
                            y_test1 = np.concatenate((y_test1,[0]))
                        elif(user_temp in intermediate):
                            y_test1 = np.concatenate((y_test1,[1]))
                        elif(user_temp in expert):
                            y_test1 = np.concatenate((y_test1,[2]))
                    else:
                        temp_x = (((features[0,j:j+1])[0])[0,:])[i]
                        temp_x1 = temp_x.reshape(1,sizeD_ts,sizeD_va)
                        X_train1 = np.concatenate((X_train1, temp_x1), axis=0)
                        user_temp = (((features[0,j:j+1])[0])[1,:])[i]
                        if(user_temp in novice):
                            y_train1 = np.concatenate((y_train1,[0]))
                        elif(user_temp in intermediate):
                            y_train1 = np.concatenate((y_train1,[1]))
                        elif(user_temp in expert):
                            y_train1 = np.concatenate((y_train1,[2]))
        
            " Delete the first column "
            X_test1 = np.delete(X_test1,np.s_[0:1], axis=0)
            y_test1 = np.delete(y_test1,np.s_[0:1])
            X_train1 = np.delete(X_train1,np.s_[0:1], axis=0)
            y_train1 = np.delete(y_train1,np.s_[0:1])

            " Shuffle the training and testing data "
            X_test, y_test = shuffle(X_test1, y_test1, random_state = random.randint(10,50))
            X_train, y_train = shuffle(X_train1, y_train1, random_state = random.randint(10,50))
    
            classes = np.unique(y_train)
            le = LabelEncoder()
            y_ind = le.fit_transform(y_train.ravel())
            recip_freq = len(y_train) / (len(le.classes_) * np.bincount(y_ind).astype(np.float64))
            class_weight = recip_freq[le.transform(classes)]
    
            " One-hot coding"
            labels_train = to_categorical(y_train)
            labels_test = to_categorical(y_test)

            ###########################################################################
            print("Ready to train")

            #######-------- auto - CNN -------------        
            
            ip = Input(shape=(sizeD_ts, sizeD_va), name='main_input')
    
            x = Conv1D(32, 7, activation='relu', padding='same', kernel_initializer='he_uniform')(ip)
            x = BatchNormalization()(x)
            x = MaxPooling1D(3)(x)
            
            x = Conv1D(16, 5, activation='relu',padding='same', kernel_initializer='he_uniform')(x)
            z = BatchNormalization()(x)
            x = MaxPooling1D(2)(x)

            x = Conv1D(16, 3, activation='relu',padding='same', kernel_initializer='he_uniform')(x)
            z = BatchNormalization()(x)
            
            fea = Flatten()(z)

            z = Conv1D(16, 3, activation='relu',padding='same', kernel_initializer='he_uniform')(z)
            z = BatchNormalization()(z)
            z = UpSampling1D(2)(z)
            
            z = Conv1D(16, 5, activation='relu',padding='same', kernel_initializer='he_uniform')(z)
            z = BatchNormalization()(z)
            z = UpSampling1D(3)(z)
            
            z = Conv1D(32, 7, activation='relu', padding='same', kernel_initializer='he_uniform')(z)
            z = BatchNormalization()(z)

            #out = Conv1D(76, 5, activation='sigmoid', padding='same', kernel_initializer='he_uniform')(z)
            out = Conv1D(240, 5, activation='sigmoid', padding='same', kernel_initializer='he_uniform')(z)

            model = Model(ip, out)

#######---------------------
            
            epochs_s = 100
            batch_s = 16
            learning_rate = 1e-3   
            weight_fn = "./weights/weights_fea_autoCNN_"+str(user)+".h5"
            model_checkpoint = ModelCheckpoint(weight_fn, verbose=1, mode='max', monitor='val_acc', save_best_only=True, save_weights_only=True)
            stop = EarlyStopping(monitor='val_loss', patience=15)
            callback_list = [model_checkpoint, stop]
            optm = Adam(lr=learning_rate)

            model.compile(optimizer=optm, loss='mean_squared_error', metrics=['accuracy'])
            model.fit(X_train, X_train, batch_size=batch_s, epochs=epochs_s, callbacks = callback_list, class_weight=class_weight, verbose=2, validation_data=(X_test, X_test))

            model.load_weights(weight_fn)
            prediction = model.predict(X_test);
            print("Skill: ", skill)
            print("Trial out:", user)
            f = open(full_output_path, 'a+')
            moFea = Model(ip, fea)
            X_train_fea = moFea.predict(X_train)
            X_test_fea = moFea.predict(X_test)
            
            svm1 = svm.SVC(kernel = 'linear')
            svm1.fit(X_train_fea, y_train)
            prediction_fea = svm1.predict(X_test_fea)
            
            y_pred_fea = prediction_fea
            cr_fea = classification_report(y_test, y_pred_fea)
            cm_fea = confusion_matrix(y_test, y_pred_fea)
            acc_fea = np.sum(y_pred_fea == y_test)/y_pred_fea.shape[0]
            acc_1_fea.append(acc_fea)
            print("-------------------- NN Fea -----------------------")
            print('---------' + skill + " Win: " + str(win) + " Shift: " + str(over) + " Trial: " + str(user) + '---------\n\n')
            print('Accuracy: ' + str(acc_fea))
            print(cr_fea)
            print(cm_fea)
            f.write("-------------------- NN Fea -----------------------")
            f.write('---------' + skill + " Win: " + str(win) + " Shift: " + str(over) + " Trial: " + str(user) + '---------\n\n')
            f.write('Accuracy: ' + str(acc_fea))
            f.write('\n\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n\n'.format(cr_fea, cm_fea))
        for i in range(0,5):
            accu_fea += acc_1_fea[i]
        accu_fea = accu_fea/5
        f.write("\n-------------------------------------------\n")
        f.write("\nAcc_fea = " + str(accu_fea))
        f.write("\n-------------------------------------------\n")
        f.close()

../results/csv_based/convAuto/1_Suture_120_60_report_AutoCNN_fea.txt already exists
../results/csv_based/convAuto/2_Suture_120_60_report_AutoCNN_fea.txt already exists
../results/csv_based/convAuto/3_Suture_120_60_report_AutoCNN_fea.txt already exists
../results/csv_based/convAuto/4_Suture_120_60_report_AutoCNN_fea.txt already exists
../results/csv_based/convAuto/5_Suture_120_60_report_AutoCNN_fea.txt already exists
../results/csv_based/convAuto/1_Knot_120_60_report_AutoCNN_fea.txt already exists
../results/csv_based/convAuto/2_Knot_120_60_report_AutoCNN_fea.txt already exists
../results/csv_based/convAuto/3_Knot_120_60_report_AutoCNN_fea.txt already exists
../results/csv_based/convAuto/4_Knot_120_60_report_AutoCNN_fea.txt already exists
../results/csv_based/convAuto/5_Knot_120_60_report_AutoCNN_fea.txt already exists
../results/csv_based/convAuto/1_Needle_120_60_report_AutoCNN_fea.txt already exists
../results/csv_based/convAuto/2_Needle_120_60_report_AutoCNN_fea.txt already exists
..